<a href="https://colab.research.google.com/github/clemvnt/training-datamining-mds/blob/issue7/20200424_07_BERLIN_final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Objective**

The Gini index measures the equality per country. The closest to 0 the index is, the most equal the country is. The world average Gini index is at 38.

**Data**

GINI INDEX

**Source**

World Bank, Development Research Group.

**Pitch**

https://drive.google.com/file/d/10PXCX0Czck8QJwhinVEoKV3MZGvlAVDC/view

**Step 1 : Import dependencies**


In [0]:
import pandas as pd
from pandas_datareader import wb
import plotly.graph_objects as go
import plotly.express as px 

**Step 2 : Get the association between the country and the ISO code**

In [0]:
countries = wb.get_countries()
countries = countries[['name', 'iso3c']]
countries.columns = ['country', 'iso3c']
countries

,country,iso3c
0,Aruba,ABW
1,Afghanistan,AFG
2,Africa,AFR
3,Angola,AGO
4,Albania,ALB
...,...,...
299,Sub-Saharan Africa excluding South Africa and ...,XZN
300,"Yemen, Rep.",YEM
301,South Africa,ZAF
302,Zambia,ZMB


**Step 3 : Get gini index indicator per country**

In [0]:
indicators = wb.download(indicator=['SI.POV.GINI'], country='all', start=1967, end=2018)
indicators.columns = ['GINI_INDEX']
indicators

GINI_INDEX
country    year            
Arab World 2018         NaN
           2017         NaN
           2016         NaN
           2015         NaN
           2014         NaN
...                     ...
Zimbabwe   1971         NaN
           1970         NaN
           1969         NaN
           1968         NaN
           1967         NaN

[13728 rows x 1 columns]

****Step 4 : Merge previous tables****

In [0]:
master_table = pd.merge(indicators.reset_index(), countries, left_on='country', right_on='country')
master_table = master_table.set_index(['country', 'iso3c', 'year'])
master_table

GINI_INDEX
country    iso3c year            
Arab World ARB   2018         NaN
                 2017         NaN
                 2016         NaN
                 2015         NaN
                 2014         NaN
...                           ...
Zimbabwe   ZWE   1971         NaN
                 1970         NaN
                 1969         NaN
                 1968         NaN
                 1967         NaN

[13624 rows x 1 columns]

**Step 5 : Pivot previous table and fill in undefined values with values from previous years**

In [0]:
pivoted_table = pd.pivot_table(master_table, index=['country', 'iso3c'], columns='year', values='GINI_INDEX')
pivoted_table = pivoted_table.ffill(axis=1)
pivoted_table

,year,1967,1969,1971,1974,1975,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
country,iso3c,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,27.0,27.0,27.0,27.0,27.0,31.7,31.7,31.7,30.6,30.6,30.6,30.0,30.0,30.0,30.0,29.0,29.0,34.6,32.9,33.7,33.2,33.2
Algeria,DZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.2,40.2,40.2,40.2,40.2,40.2,40.2,35.3,35.3,35.3,35.3,35.3,35.3,35.3,35.3,35.3,35.3,35.3,35.3,35.3,35.3,35.3,35.3,27.6,27.6,27.6,27.6,27.6,27.6,27.6,27.6
Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,52.0,52.0,52.0,52.0,52.0,52.0,52.0,42.7,42.7,42.7,42.7,42.7,42.7,42.7,42.7,42.7,42.7,51.3
Argentina,ARG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.8,40.8,40.8,40.8,40.8,40.8,42.8,45.3,45.3,45.3,45.3,46.8,45.5,44.9,45.9,48.9,49.5,49.1,50.7,49.8,51.1,53.3,53.8,51.2,48.6,48.0,46.7,46.6,45.3,44.1,44.5,42.7,41.4,41.0,41.7,41.7,42.0,41.2,41.4
Armenia,ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.2,36.2,35.4,34.8,33.0,37.5,36.0,29.7,31.2,29.2,28.0,30.0,29.4,29.6,30.6,31.5,32.4,32.5,33.6,34.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,VNM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.7,35.7,35.7,35.7,35.7,35.7,35.4,35.4,35.4,35.4,37.0,37.0,36.8,36.8,35.8,35.8,35.6,35.6,39.3,39.3,35.6,35.6,34.8,34.8,35.3,35.3,35.7
West Bank and Gaza,PSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,34.7,34.0,35.6,35.6,34.5,35.3,34.4,34.4,34.4,34.4,34.4,33.7,33.7,33.7
"Yemen, Rep.",YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,35.0,35.0,35.0,35.0,35.0,35.0,34.7,34.7,34.7,34.7,34.7,34.7,34.7,34.7,34.7,36.7,36.7,36.7,36.7,36.7


**Step 5 : Show a map of gini index per country over the years (from 1969 to 2018)**

In [0]:
pivoted_table = pd.pivot_table(master_table, index=['country', 'iso3c'], columns='year', values='GINI_INDEX')
pivoted_table = pivoted_table.ffill(axis=1)
countries = list(pivoted_table.index.get_level_values(0))
locations = list(pivoted_table.index.get_level_values(1))

data = []
steps = []
i = 0
for year in pivoted_table.columns:
  data.append(dict(
    type='choropleth',
    name='',
    locations=locations,
    z=pivoted_table[year],
    hovertext=countries,
    colorscale=px.colors.sequential.Reds,
    visible=year=='2018'
  ))
  
  step = dict(
    method='restyle',
    args=['visible', [False] * len(pivoted_table.columns)],
    label=year)
  step['args'][1][i] = True
  steps.append(step)

  i = i + 1

layout = go.Layout(
  title=dict(
    text='Evolution of the gini index from 1969 to 2018', 
    x=0.5,
    font=dict(
      size=21,
    )
  ),
  sliders=[dict(steps=steps, active=len(data) - 1)],
  annotations=[dict(
    text='Updated in 2018 from The World Bank',
    showarrow=False,
    x=1,
    y=-0.05
  )],
  autosize=True,
  height=800
)

fig = go.Figure(data, layout)
fig

**Step 6 : Export HTML**

In [0]:
fig.write_html("file.html")